In [30]:
import numpy as np
import pandas as pd
import itertools

%load_ext autoreload
%autoreload 2

pindir="../pin_experiments"
configs = ["base", "C1", "C2", "C3", "C4"]

# Linux dvfs policies: https://www.kernel.org/doc/Documentation/cpu-freq/governors.txt
policies = ["ondemand"]

# total time to run for, in ms
times = [2600000]

# diff flink rates
rates = [100000, 200000, 300000, 400000]

# number of mappers
mappers = [4, 8, 12, 16]

# not exploring different combo for these yet
sources = [16] # num of sources
sinks = [16] # num of sinks

df_dict = {
    "query" : [],
    "config" : [],
    "nsource" : [],
    "nmapper" : [],
    "nsink" : [],
    "RPS" : [],
    "pkgPower" : [],
    "pkgPowerstd" : [],
    "dramPower" : [],
    "dramPowerstd" : [],
    "SourceOut" : [],
    "POLL" : [],
    "C1" : [],
    "C1E" : [],
    "C3" : [],
    "C6" : [],
    "cpu_user" : [],
    "cpu0_user": [],
    "cpu1_user": [],
    "cpu2_user": [],
    "cpu3_user": [],
    "cpu4_user": [],
    "cpu5_user": [],
    "cpu6_user": [],
    "cpu7_user": [],
    "cpu8_user": [],
    "cpu9_user": [],
    "cpu10_user": [],
    "cpu11_user": [],
    "cpu12_user": [],
    "cpu13_user": [],
    "cpu14_user": [],
    "cpu15_user": [],
    "cpu_idle" : [],
    "cpu0_idle": [],
    "cpu1_idle": [],
    "cpu2_idle": [],
    "cpu3_idle": [],
    "cpu4_idle": [],
    "cpu5_idle": [],
    "cpu6_idle": [],
    "cpu7_idle": [],
    "cpu8_idle": [],
    "cpu9_idle": [],
    "cpu10_idle": [],
    "cpu11_idle": [],
    "cpu12_idle": [],
    "cpu13_idle": [],
    "cpu14_idle": [],
    "cpu15_idle": []
}

def resetdict():
    global df_dict
    
    df_dict = {
    "query" : [],
    "config" : [],
    "nsource" : [],
    "nmapper" : [],
    "nsink" : [],
    "RPS" : [],
    "pkgPower" : [],
    "pkgPowerstd" : [],
    "dramPower" : [],
    "dramPowerstd" : [],
    "SourceOut" : [],
    "POLL" : [],
    "C1" : [],
    "C1E" : [],
    "C3" : [],
    "C6" : [],
    "cpu_user" : [],
    "cpu0_user": [],
    "cpu1_user": [],
    "cpu2_user": [],
    "cpu3_user": [],
    "cpu4_user": [],
    "cpu5_user": [],
    "cpu6_user": [],
    "cpu7_user": [],
    "cpu8_user": [],
    "cpu9_user": [],
    "cpu10_user": [],
    "cpu11_user": [],
    "cpu12_user": [],
    "cpu13_user": [],
    "cpu14_user": [],
    "cpu15_user": [],
    "cpu_idle" : [],
    "cpu0_idle": [],
    "cpu1_idle": [],
    "cpu2_idle": [],
    "cpu3_idle": [],
    "cpu4_idle": [],
    "cpu5_idle": [],
    "cpu6_idle": [],
    "cpu7_idle": [],
    "cpu8_idle": [],
    "cpu9_idle": [],
    "cpu10_idle": [],
    "cpu11_idle": [],
    "cpu12_idle": [],
    "cpu13_idle": [],
    "cpu14_idle": [],
    "cpu15_idle": []
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
def run(policy, rate, timems, mapper, source, sink):
    
    #"query1_cores16_frate100000_2600000_fbuff-1_itr1_ondemanddvfs1_source16_mapper4_sink16_repeat0"
    querydir = f"query1_cores16_frate{rate}_{timems}_fbuff-1_itr1_{policy}dvfs1_source{source}_mapper{mapper}_sink{sink}_repeat0"
    
    for c in configs:
        fname = f"{pindir}/{querydir}/{c}"
    
        df_dict['config'].append(c)
        df_dict['query'].append('query1')
        df_dict['nsource'].append(source)
        df_dict['nmapper'].append(mapper)
        df_dict['nsink'].append(sink)
        df_dict['RPS'].append(rate)
    
        # Power
        pkgs=[]
        dram=[]
        with open(f"{fname}/rapl.log") as f:
            lines = f.readlines()
        for line in lines:
            if float(line.strip().split(' ')[0]) > 0.0:
                pkgs.append(float(line.strip().split(' ')[0]))
            if float(line.strip().split(' ')[1]) > 0.0:
                dram.append(float(line.strip().split(' ')[1]))
        #print(f"CPU pkg: {np.mean(np.array(pkgs))} {np.std(np.array(pkgs))}")
        #print(f"DRAM pkg: {np.mean(np.array(dram))} {np.std(np.array(dram))}")
        df_dict['pkgPower'].append(np.mean(np.array(pkgs)))
        df_dict['pkgPowerstd'].append(np.std(np.array(pkgs)))
        df_dict['dramPower'].append(np.mean(np.array(dram)))
        df_dict['dramPowerstd'].append(np.std(np.array(dram)))
    
        # Source records out
        kwlist={'numRecordsOutPerSecond':[]}
    
        with open(f"{fname}/Source.0") as f:
            lines = f.read()
        for lc in lines.split('; '):
            for kw in kwlist.keys():
                if(kw in lc):
                    ldict=eval(lc.replace('[','').replace(']',''))
                    sourceout = float(ldict['value'])
        #print(f"Source out: {sourceout}")
        df_dict['SourceOut'].append(sourceout)
        
        # idle stats
        statsSTART = pd.read_csv(f"{fname}/statsSTART.csv", names=['POLL', 'C1', 'C1E', 'C3', 'C6', 
                                                           'RXP', 'RXB', 'TXP', 'TXB', 'ERXP', 'ERXB', 'ETXP', 'ETXB'])
        statsEND = pd.read_csv(f"{fname}/statsEND.csv", names=['POLL', 'C1', 'C1E', 'C3', 'C6', 
                                                           'RXP', 'RXB', 'TXP', 'TXB', 'ERXP', 'ERXB', 'ETXP', 'ETXB'])
        statsdf = pd.concat([statsSTART, statsEND])
        statsdf = statsdf.diff()
        statsdf.dropna(inplace=True)
        df_dict['POLL'].append(statsdf['POLL'].values[0])
        df_dict['C1'].append(statsdf['C1'].values[0])
        df_dict['C1E'].append(statsdf['C1E'].values[0])
        df_dict['C3'].append(statsdf['C3'].values[0])
        df_dict['C6'].append(statsdf['C6'].values[0])
        #print(statsdf[['POLL', 'C1', 'C3', 'C1E', 'C6']])
    
        # /proc/stat
        with open(f"{fname}/procstatsSTART.csv") as f:
            lines = f.readlines()
    
        procstats={}
        for line in lines:
            if 'cpu' in line:
                stats = line.strip().split(' ')
                #print(stats)
                tmp = []
                for i in range(1, len(stats)):
                    if len(stats[i]) > 0:
                        tmp.append(int(stats[i]))
                procstats[stats[0]] = tmp
                #print(stats[0], len(tmp))
        
        with open(f"{fname}/procstatsEND.csv") as f:
            lines = f.readlines()
        for line in lines:
            if 'cpu' in line:
                stats = line.strip().split(' ')
                tmp = []
                #print(stats)
                for i in range(1, len(stats)):
                    if len(stats[i]) > 0:
                        tmp.append(int(stats[i]))
                for i in range(0, len(tmp)):
                    procstats[stats[0]][i] = tmp[i] - procstats[stats[0]][i]
        #print(procstats)  
    
        for k, v in procstats.items():
            df_dict[f"{k}_user"].append(v[0])
            df_dict[f"{k}_idle"].append(v[3])
    
    #dd1 = pd.DataFrame(df_dict)
    #print(dd1.columns)
    #return dd1


In [32]:
resetdict()

# Generate combinations of items from the 
combinations = list(itertools.product(policies, rates, times,  mappers, sources, sinks))

# Print the combinations
for combo in combinations:
    
    policy, rate, timems, mapper, source, sink = combo
    run(policy, rate, timems, mapper, source, sink)


# 100K RPS

In [33]:
dd1 = pd.DataFrame(df_dict)

dd1[(dd1.RPS == 100000)][['query', 'nsource', 'nmapper', 'nsink', 'RPS', 'config', 'pkgPower', 'dramPower', 'SourceOut', 'POLL', 'C1', 'C1E', 'C3', 'C6', 'cpu_user', 'cpu_idle']]

,query,nsource,nmapper,nsink,RPS,config,pkgPower,dramPower,SourceOut,POLL,C1,C1E,C3,C6,cpu_user,cpu_idle
0,query1,16,4,16,100000,base,38.321666,4.115080,99207.466667,887.0,273557.0,818317.0,0.0,312957.0,43063,144907
1,query1,16,4,16,100000,C1,39.286580,4.122152,100537.283333,916.0,287756.0,792842.0,0.0,292624.0,43080,144760
2,query1,16,4,16,100000,C2,38.719867,4.185240,99965.416667,1185.0,265587.0,846395.0,0.0,333490.0,44299,143638
3,query1,16,4,16,100000,C3,39.136637,4.182240,99941.583333,967.0,257281.0,841095.0,0.0,350464.0,44182,143751
4,query1,16,4,16,100000,C4,39.375024,4.308017,99725.900000,842.0,254929.0,812607.0,0.0,354106.0,43196,144578
5,query1,16,8,16,100000,base,30.161913,3.797146,100000.000000,783.0,339078.0,483512.0,0.0,207403.0,63034,125228
6,query1,16,8,16,100000,C1,31.810722,3.393163,100000.000000,324.0,78261.0,128678.0,0.0,123914.0,40777,148299
7,query1,16,8,16,100000,C2,31.870026,3.446460,100000.000000,412.0,117391.0,163837.0,0.0,130521.0,40671,148522
8,query1,16,8,16,100000,C3,30.437293,3.239362,100000.000000,568.0,342735.0,200982.0,0.0,161063.0,40827,148057
9,query1,16,8,16,100000,C4,30.080617,3.213221,100000.000000,459.0,338696.0,191364.0,0.0,141952.0,41345,147338


# 200K RPS

In [34]:
dd1[(dd1.RPS == 200000)][['query', 'nsource', 'nmapper', 'nsink', 'RPS', 'config', 'pkgPower', 'dramPower', 'SourceOut', 'POLL', 'C1', 'C1E', 'C3', 'C6', 'cpu_user', 'cpu_idle']]

,query,nsource,nmapper,nsink,RPS,config,pkgPower,dramPower,SourceOut,POLL,C1,C1E,C3,C6,cpu_user,cpu_idle
20,query1,16,4,16,200000,base,42.897167,4.307382,107067.450000,872.0,272613.0,884459.0,0.0,324974.0,46518,141289
21,query1,16,4,16,200000,C1,44.534584,4.279030,108060.266667,1054.0,311506.0,923513.0,0.0,288595.0,47064,140641
22,query1,16,4,16,200000,C2,43.661390,4.268479,107778.750000,1485.0,305559.0,953852.0,0.0,277338.0,47120,140824
23,query1,16,4,16,200000,C3,43.690941,4.251582,108271.916667,886.0,286583.0,899848.0,0.0,317802.0,46897,140817
24,query1,16,4,16,200000,C4,52.697815,4.404407,119521.566667,1009.0,374069.0,1031245.0,0.0,278113.0,49529,138144
25,query1,16,8,16,200000,base,60.237830,5.438348,199917.316667,2207.0,485711.0,729659.0,0.0,426590.0,86212,99533
26,query1,16,8,16,200000,C1,57.161432,5.431484,199919.700000,446.0,159023.0,280605.0,0.0,138211.0,82978,103219
27,query1,16,8,16,200000,C2,56.619391,5.324570,199949.150000,590.0,194772.0,321099.0,0.0,138483.0,82943,103513
28,query1,16,8,16,200000,C3,62.029015,5.345487,200000.000000,1187.0,549923.0,791055.0,0.0,405957.0,86057,99656
29,query1,16,8,16,200000,C4,57.283338,5.422119,199597.216667,964.0,460815.0,747743.0,0.0,409818.0,84636,100805


# 300K RPS

In [42]:
dd1[(dd1.RPS == 300000)][['query', 'nsource', 'nmapper', 'nsink', 'RPS', 'config', 'pkgPower', 'dramPower', 'SourceOut', 'POLL', 'C1', 'C1E', 'C3', 'C6', 'cpu_user', 'cpu_idle']]

,query,nsource,nmapper,nsink,RPS,config,pkgPower,dramPower,SourceOut,POLL,C1,C1E,C3,C6,cpu_user,cpu_idle
40,query1,16,4,16,300000,base,43.568682,4.358904,83245.383333,856.0,163146.0,764846.0,0.0,389888.0,43290,145166
41,query1,16,4,16,300000,C1,44.241170,4.349756,82087.400000,828.0,169462.0,833934.0,0.0,354367.0,44066,144535
42,query1,16,4,16,300000,C2,45.002928,4.458047,84795.733333,995.0,179151.0,802824.0,0.0,359296.0,44157,144330
43,query1,16,4,16,300000,C3,44.280727,4.455441,83830.966667,794.0,168813.0,772917.0,0.0,374599.0,44260,144176
44,query1,16,4,16,300000,C4,48.794809,4.514778,84607.600000,704.0,157330.0,767452.0,0.0,373352.0,44505,143784
45,query1,16,8,16,300000,base,62.771892,5.402685,222370.650000,1204.0,605484.0,748665.0,0.0,351155.0,91843,93801
46,query1,16,8,16,300000,C1,65.239710,5.522813,220348.316667,164.0,62441.0,70639.0,0.0,67720.0,88229,97890
47,query1,16,8,16,300000,C2,63.620390,5.512686,219919.966667,273.0,98147.0,107991.0,0.0,70170.0,87989,98147
48,query1,16,8,16,300000,C3,62.755239,5.377245,218703.483333,1221.0,657179.0,760946.0,0.0,336147.0,91622,93784
49,query1,16,8,16,300000,C4,61.178810,5.514935,236036.366667,1042.0,557447.0,715039.0,0.0,255646.0,95968,89310


# 400K RPS

In [43]:
dd1[(dd1.RPS == 400000)][['query', 'nsource', 'nmapper', 'nsink', 'RPS', 'config', 'pkgPower', 'dramPower', 'SourceOut', 'POLL', 'C1', 'C1E', 'C3', 'C6', 'cpu_user', 'cpu_idle']]

,query,nsource,nmapper,nsink,RPS,config,pkgPower,dramPower,SourceOut,POLL,C1,C1E,C3,C6,cpu_user,cpu_idle
60,query1,16,4,16,400000,base,42.312500,4.337834,105666.166667,895.0,268454.0,846379.0,0.0,324012.0,46373,141551
61,query1,16,4,16,400000,C1,44.319776,4.335690,107562.116667,1117.0,301186.0,911866.0,0.0,280690.0,47194,140660
62,query1,16,4,16,400000,C2,44.882516,4.353695,109549.416667,1458.0,317871.0,936622.0,0.0,295374.0,47256,140539
63,query1,16,4,16,400000,C3,43.337260,4.294324,105810.016667,892.0,279950.0,881701.0,0.0,325715.0,46635,141170
64,query1,16,4,16,400000,C4,53.178957,4.481298,120298.750000,958.0,375734.0,987423.0,0.0,278636.0,49710,137951
65,query1,16,8,16,400000,base,64.273896,5.577702,221305.466667,1104.0,567251.0,713593.0,0.0,361802.0,90999,94305
66,query1,16,8,16,400000,C1,66.756667,5.502556,218747.033333,172.0,54350.0,61270.0,0.0,65880.0,88544,97675
67,query1,16,8,16,400000,C2,63.589306,5.516117,216940.900000,309.0,82433.0,91244.0,0.0,71806.0,87945,98158
68,query1,16,8,16,400000,C3,64.640175,5.540026,221460.633333,1098.0,610653.0,748103.0,0.0,347203.0,91068,94122
69,query1,16,8,16,400000,C4,59.717316,5.532387,224337.466667,915.0,478749.0,689531.0,0.0,329307.0,91860,93016


# Filter out configurations that did not meet RPS rate

In [51]:

dd1[(dd1.RPS == 100000) & ((dd1.SourceOut/dd1.RPS) >= 0.9)][['query', 'nsource', 'nmapper', 'nsink', 'RPS', 'config', 'pkgPower', 'dramPower', 'SourceOut', 'POLL', 'C1', 'C1E', 'C3', 'C6', 'cpu_user', 'cpu_idle']]


,query,nsource,nmapper,nsink,RPS,config,pkgPower,dramPower,SourceOut,POLL,C1,C1E,C3,C6,cpu_user,cpu_idle
0,query1,16,4,16,100000,base,38.321666,4.115080,99207.466667,887.0,273557.0,818317.0,0.0,312957.0,43063,144907
1,query1,16,4,16,100000,C1,39.286580,4.122152,100537.283333,916.0,287756.0,792842.0,0.0,292624.0,43080,144760
2,query1,16,4,16,100000,C2,38.719867,4.185240,99965.416667,1185.0,265587.0,846395.0,0.0,333490.0,44299,143638
3,query1,16,4,16,100000,C3,39.136637,4.182240,99941.583333,967.0,257281.0,841095.0,0.0,350464.0,44182,143751
4,query1,16,4,16,100000,C4,39.375024,4.308017,99725.900000,842.0,254929.0,812607.0,0.0,354106.0,43196,144578
5,query1,16,8,16,100000,base,30.161913,3.797146,100000.000000,783.0,339078.0,483512.0,0.0,207403.0,63034,125228
6,query1,16,8,16,100000,C1,31.810722,3.393163,100000.000000,324.0,78261.0,128678.0,0.0,123914.0,40777,148299
7,query1,16,8,16,100000,C2,31.870026,3.446460,100000.000000,412.0,117391.0,163837.0,0.0,130521.0,40671,148522
8,query1,16,8,16,100000,C3,30.437293,3.239362,100000.000000,568.0,342735.0,200982.0,0.0,161063.0,40827,148057
9,query1,16,8,16,100000,C4,30.080617,3.213221,100000.000000,459.0,338696.0,191364.0,0.0,141952.0,41345,147338


In [48]:
dd1[(dd1.RPS == 200000) & ((dd1.SourceOut/dd1.RPS) >= 0.9)][['query', 'nsource', 'nmapper', 'nsink', 'RPS', 'config', 'pkgPower', 'dramPower', 'SourceOut', 'POLL', 'C1', 'C1E', 'C3', 'C6', 'cpu_user', 'cpu_idle']]

,query,nsource,nmapper,nsink,RPS,config,pkgPower,dramPower,SourceOut,POLL,C1,C1E,C3,C6,cpu_user,cpu_idle
25,query1,16,8,16,200000,base,60.237830,5.438348,199917.316667,2207.0,485711.0,729659.0,0.0,426590.0,86212,99533
26,query1,16,8,16,200000,C1,57.161432,5.431484,199919.700000,446.0,159023.0,280605.0,0.0,138211.0,82978,103219
27,query1,16,8,16,200000,C2,56.619391,5.324570,199949.150000,590.0,194772.0,321099.0,0.0,138483.0,82943,103513
28,query1,16,8,16,200000,C3,62.029015,5.345487,200000.000000,1187.0,549923.0,791055.0,0.0,405957.0,86057,99656
29,query1,16,8,16,200000,C4,57.283338,5.422119,199597.216667,964.0,460815.0,747743.0,0.0,409818.0,84636,100805
30,query1,16,12,16,200000,base,51.337591,5.093639,200039.483333,1253.0,560054.0,490648.0,0.0,325209.0,106088,79760
31,query1,16,12,16,200000,C1,55.822055,5.377656,200010.200000,169.0,48111.0,92670.0,0.0,98615.0,83250,102829
32,query1,16,12,16,200000,C2,55.323666,5.322400,200052.950000,358.0,51677.0,93909.0,0.0,104147.0,83393,102720
33,query1,16,12,16,200000,C3,51.733640,4.945165,199330.516667,962.0,306266.0,355485.0,0.0,391348.0,87544,98470
34,query1,16,12,16,200000,C4,55.023367,5.166842,199986.133333,820.0,658780.0,487976.0,0.0,268083.0,84045,101223


In [49]:
dd1[(dd1.RPS == 300000) & ((dd1.SourceOut/dd1.RPS) >= 0.9)][['query', 'nsource', 'nmapper', 'nsink', 'RPS', 'config', 'pkgPower', 'dramPower', 'SourceOut', 'POLL', 'C1', 'C1E', 'C3', 'C6', 'cpu_user', 'cpu_idle']]

,query,nsource,nmapper,nsink,RPS,config,pkgPower,dramPower,SourceOut,POLL,C1,C1E,C3,C6,cpu_user,cpu_idle
50,query1,16,12,16,300000,base,72.290428,6.101486,294436.516667,1416.0,600330.0,575901.0,0.0,423527.0,128366,55702
53,query1,16,12,16,300000,C3,72.674455,6.173110,297530.166667,1339.0,629307.0,536825.0,0.0,416274.0,128762,55413
55,query1,16,16,16,300000,base,73.625076,6.070509,299892.816667,2288.0,1234736.0,1414167.0,0.0,331089.0,131368,50964
58,query1,16,16,16,300000,C3,72.955993,5.913515,300077.483333,2211.0,1237942.0,1413563.0,0.0,297028.0,130403,52328


In [50]:
dd1[(dd1.RPS == 400000) & ((dd1.SourceOut/dd1.RPS) >= 0.9)][['query', 'nsource', 'nmapper', 'nsink', 'RPS', 'config', 'pkgPower', 'dramPower', 'SourceOut', 'POLL', 'C1', 'C1E', 'C3', 'C6', 'cpu_user', 'cpu_idle']]

,query,nsource,nmapper,nsink,RPS,config,pkgPower,dramPower,SourceOut,POLL,C1,C1E,C3,C6,cpu_user,cpu_idle
75,query1,16,16,16,400000,base,86.944220,6.737450,399826.483333,1661.0,838889.0,560562.0,0.0,114233.0,163372,18437
78,query1,16,16,16,400000,C3,86.906754,6.509903,400016.350000,1696.0,907355.0,585067.0,0.0,119988.0,163841,18661
